In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

122880/118774 [===============================] - 0s 1us/step


In [3]:
BATCH_SIZE = 64

In [4]:
train_df = pd.read_csv(train_file_path, sep='\t', names=['label', 'text'])
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
train_df.tail(2)

,label,text
4177,1,-pls stop bootydelious (32/f) is inviting you ...
4178,0,tell my bad character which u dnt lik in me. ...


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_df.text.values, train_df.label.values))
for ex in train_dataset.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'you can never do nothing'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'mum say we wan to go then go... then she can shun bian watch da glass exhibition...'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'never y lei... i v lazy... got wat? dat day \xc3\xbc send me da url cant work one...'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [6]:
test_df = pd.read_csv(test_file_path, sep='\t', names=['label', 'text'])
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})
test_df.tail(2)

,label,text
1390,1,ur cash-balance is currently 500 pounds - to m...
1391,1,not heard from u4 a while. call 4 rude chat pr...


In [7]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_df.text.values, test_df.label.values))
for ex in test_dataset.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'i am in hospital da. . i will return home in evening'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"not much, just some textin'. how bout you?">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"i probably won't eat at all today. i think i'm gonna pop. how was your weekend? did u miss me?">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'don\xe2\x80\x98t give a flying monkeys wot they think and i certainly don\xe2\x80\x98t mind. any friend of mine and all that!'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'who are you seeing?'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [8]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in train_dataset:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)
for text_tensor, _ in test_dataset:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

8741

In [9]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [10]:
example_text = next(iter(train_dataset))[0].numpy()
print(example_text)

b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'


In [11]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[3571, 5701, 7726, 2008, 2512, 1986, 1598, 6007, 3038, 4607, 4691, 4426, 4684, 6962, 8048, 4607, 2299, 2947, 4684, 6154, 2301, 3374, 3038, 7927, 4808, 311, 4684, 2857, 3339, 2008, 4823, 6953]


In [12]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [13]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


encoded_train_dataset = train_dataset.map(encode_map_fn)
encoded_test_dataset = test_dataset.map(encode_map_fn)

for ex in encoded_train_dataset.take(5):
  print(ex)

(<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3571, 5701, 7726, 2008, 2512, 1986, 1598, 6007, 3038, 4607, 4691,
       4426, 4684, 6962, 8048, 4607, 2299, 2947, 4684, 6154, 2301, 3374,
       3038, 7927, 4808,  311, 4684, 2857, 3339, 2008, 4823, 6953],
      dtype=int64)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=int64, numpy=array([7813, 1463, 4224, 8092, 4020], dtype=int64)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(24,), dtype=int64, numpy=
array([2947, 4607,  156, 8048, 4884, 2016, 3348, 5047, 8048, 4684, 1317,
        825, 5671, 5275, 5659, 5314, 2046, 3858, 4607, 1007,  908, 7439,
       4092, 5038], dtype=int64)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int64, numpy=
array([3737, 3559, 6502,  626,  916, 1632, 1859, 1632, 1859, 6086, 1463,
       3778,  115, 7193, 5275, 3396, 7973], dtype=int64)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(18,), dtype=int64

In [14]:
train_data = encoded_train_dataset.padded_batch(BATCH_SIZE)

test_data = encoded_test_dataset.padded_batch(BATCH_SIZE)

for ex in train_data.take(2):
  print(ex)

(<tf.Tensor: shape=(64, 97), dtype=int64, numpy=
array([[3571, 5701, 7726, ...,    0,    0,    0],
       [7813, 1463, 4224, ...,    0,    0,    0],
       [2947, 4607,  156, ...,    0,    0,    0],
       ...,
       [1993, 6962, 3559, ...,    0,    0,    0],
       [7935, 7935, 2058, ...,    0,    0,    0],
       [7193, 6297, 4684, ...,    0,    0,    0]], dtype=int64)>, <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)>)
(<tf.Tensor: shape=(64, 40), dtype=int64, numpy=
array([[4684,   26, 5712, ...,    0,    0,    0],
       [1463, 7813,  125, ...,    0,    0,    0],
       [7968,  311, 7844, ...,    0,    0,    0],
       ...,
       [5282, 2755, 5622, ...,    0,    0,    0],
       [5882, 1184, 5975, ...,    0,    0,    0],
       [  42, 1536, 4684, ...,  

In [15]:
vocab_size += 2
vocab_size

8743

In [16]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          559552    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 633,921
Trainable params: 633,921
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [19]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
66/66 [==============================] - 7s 106ms/step - loss: 0.3215 - accuracy: 0.8847 - val_loss: 0.0975 - val_accuracy: 0.9734
Epoch 2/3
66/66 [==============================] - 4s 65ms/step - loss: 0.0457 - accuracy: 0.9873 - val_loss: 0.0464 - val_accuracy: 0.9856
Epoch 3/3
66/66 [==============================] - 4s 65ms/step - loss: 0.0160 - accuracy: 0.9957 - val_loss: 0.0447 - val_accuracy: 0.9885


In [20]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded_text = encoder.encode(pred_text)
  pred = model.predict([encoded_text])[0][0]
  label = 'ham'
  if pred > 0.5:
    label = 'spam'

  return [pred, label]

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

[0.1017605, 'ham']


In [21]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
